
# 📊 Phân tích theo **thể loại & ca sĩ** (dùng cột thật trong `dataset_cleaned.csv`)

**Cột sử dụng:**  
- `track_genre` (thể loại)  
- `artists` (ca sĩ)  
- `track_name` (tên bài hát)  
- `popularity` (độ phổ biến)

**Mục tiêu:**  
- Thống kê số lượng bài hát theo **thể loại** và **ca sĩ**.  
- Tính **độ phổ biến trung bình** cho từng nhóm.  
- Vẽ **biểu đồ** minh hoạ Top 10.  

> Ghi chú: chỉ dùng **matplotlib** (không seaborn). Đặt `dataset_cleaned.csv` cùng thư mục với notebook.
---


In [ ]:

# %% [code] Thiết lập & đọc dữ liệu
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv("C:/Users/minht/Dropbox/PC/Downloads/dataset_cleaned.csv")
print("Shape:", df.shape)
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'data_cleaned.csv'


## 1) Kiểm tra cột và tiền xử lý nhẹ


In [ ]:

# %% [code] Kiểm tra cột bắt buộc
required_cols = ["track_name", "artists", "track_genre", "popularity"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Thiếu cột: {missing}. Cần có {required_cols}")

# Ép popularity về số & loại bỏ NA quan trọng
df["popularity"] = pd.to_numeric(df["popularity"], errors="coerce")
before = len(df)
df = df.dropna(subset=["artists", "track_genre", "popularity"])
print("Dropped:", before - len(df), "rows")
df.sample(min(5, len(df)))



## 2) Phân tích **thể loại (track_genre)**
### 2.1 Thống kê số lượng bài hát & độ phổ biến trung bình


In [ ]:

# %% [code] GroupBy theo thể loại
genre_stats = (
    df.groupby("track_genre")
      .agg(song_count=("track_name", "count"), avg_popularity=("popularity", "mean"))
      .reset_index()
      .sort_values("song_count", ascending=False)
)
genre_stats.head(10)



### 2.2 Biểu đồ: Top 10 thể loại nhiều bài hát nhất


In [ ]:

# %% [code]
top_genres_count = genre_stats.nlargest(10, "song_count").iloc[::-1]
plt.figure(figsize=(10,6))
plt.barh(top_genres_count["track_genre"], top_genres_count["song_count"])
plt.title("Top 10 thể loại có nhiều bài hát nhất")
plt.xlabel("Số lượng bài hát")
plt.ylabel("Thể loại (track_genre)")
plt.tight_layout()
plt.show()



### 2.3 Biểu đồ: Top 10 thể loại có độ phổ biến trung bình cao nhất


In [ ]:

# %% [code]
top_genres_pop = genre_stats.nlargest(10, "avg_popularity").iloc[::-1]
plt.figure(figsize=(10,6))
plt.barh(top_genres_pop["track_genre"], top_genres_pop["avg_popularity"])
plt.title("Top 10 thể loại phổ biến nhất (độ phổ biến trung bình)")
plt.xlabel("Độ phổ biến trung bình (popularity)")
plt.ylabel("Thể loại (track_genre)")
plt.tight_layout()
plt.show()



> **Nhận xét (Genre):**  
> - Một số `track_genre` có **nhiều bài hát** nhất phản ánh mức độ phủ rộng của thể loại đó trong dataset.  
> - **Độ phổ biến trung bình (avg_popularity)** cho thấy mức độ yêu thích — có thể khác thứ hạng theo số lượng.  
> - Nếu cần gợi ý cho người dùng mới, có thể ưu tiên các thể loại có `avg_popularity` cao.



## 3) Phân tích **ca sĩ (artists)**
### 3.1 Thống kê số lượng bài hát & độ phổ biến trung bình


In [ ]:

# %% [code]
artist_stats = (
    df.groupby("artists")
      .agg(song_count=("track_name", "count"), avg_popularity=("popularity", "mean"))
      .reset_index()
      .sort_values("song_count", ascending=False)
)
artist_stats.head(10)



### 3.2 Biểu đồ: Top 10 ca sĩ có nhiều bài hát nhất


In [ ]:

# %% [code]
top_artists_count = artist_stats.nlargest(10, "song_count").iloc[::-1]
plt.figure(figsize=(10,6))
plt.barh(top_artists_count["artists"], top_artists_count["song_count"])
plt.title("Top 10 ca sĩ có nhiều bài hát nhất")
plt.xlabel("Số lượng bài hát")
plt.ylabel("Ca sĩ (artists)")
plt.tight_layout()
plt.show()



### 3.3 Biểu đồ: Top 10 ca sĩ có độ phổ biến trung bình cao nhất


In [ ]:

# %% [code]
top_artists_pop = artist_stats.nlargest(10, "avg_popularity").iloc[::-1]
plt.figure(figsize=(10,6))
plt.barh(top_artists_pop["artists"], top_artists_pop["avg_popularity"])
plt.title("Top 10 ca sĩ phổ biến nhất (độ phổ biến trung bình)")
plt.xlabel("Độ phổ biến trung bình (popularity)")
plt.ylabel("Ca sĩ (artists)")
plt.tight_layout()
plt.show()



> **Nhận xét (Artist):**  
> - Nghệ sĩ có **nhiều bài hát** thường là những người phát hành nhiều hoặc xuất hiện trong nhiều album/playlist.  
> - Nghệ sĩ có **avg_popularity** cao cho thấy mức độ yêu thích mạnh dù số bài có thể không nhiều.  
> - Hai chiều "số lượng vs yêu thích" hữu ích để xây dựng gợi ý theo nghệ sĩ.



## 4) Kết luận
- Tồn tại chênh lệch giữa **độ phủ (song_count)** và **mức độ yêu thích (avg_popularity)** ở cả thể loại và ca sĩ.  
- Kết quả giúp:
  - Gợi ý bài hát trong **cùng `track_genre`** có avg_popularity cao.  
  - Gợi ý **nghệ sĩ tương tự** dựa trên thống kê theo `artists`.  

> Bạn có thể chép phần nhận xét/kết luận vào báo cáo.
